# 一步一步建立神经网络

欢迎来到第四周第一部分的练习！之前你已经训练了单隐层的双层神经网络

这周你将会建立一个多层神经网络(层数可自定义)

- 在本次课程中，你将会实现了建立神经网络需要的所有函数
- 在下一个作业中，你将用这些函数来建立一个图像分类的深度神经网络


**完成这次任务后，你会学得**

- 使用想ReLU这样的非线性单元来提高你的模型
- 建立一个深度神经网络 (超过1个隐含层)
- 实现一个易用的神经网络类

**注释**:
- 上标$[l]$表示和第$l^{th}$层相关的量
    - 例: $a^{[L]}$ 是第 $L^{th}$层的激活值 $W^{[L]}$和$b^{[L]}$是第$L^{th}$层的参数
- 上标$(i)$表示和第$i^{th}$个样本相关的量
    - 例: $x^{(i)}$是第$i^{th}$训练样本
- 下标$i$表示第 $i^{th}$个向量的输入
    - 例: $a^{[l]}_i$ 是第$l^{th}$层第$i^{th}$的激活值

让我们开始吧！

## 目录

- [1 - 导包](#1)
- [2 - 概要](#2)
- [3 - 初始化](#3)
  - [3.1 - 2层神经网络](#3-1)
    - [练习 1 - 初始化参数](#ex-1)
  - [3.2 - L层神经网络](#3-2)
    - [练习 2 - 初始化参数深度](#ex-2)
- [4 - 前向传播模块](#4)
  - [4.1 - 前向传播-线性部分](#4-1)
    - [练习 3 - 前向传播-线性部分](#ex-3)
  - [4.2 - 前向传播-激活部分](#4-2)
    - [练习 4 - 前向传播-线性+激活部分](#ex-4)
  - [4.3 - L层模型](#4-3)
    - [练习 5 - L层模型的前向传播](#ex-5)
- [5 - 代价函数](#5)
  - [练习 6 - 计算cost](#ex-6)
- [6 - 后向传播模块](#6)
  - [6.1 - 线性后向传播](#6-1)
    - [练习 7 - 线性后向传播](#ex-7)
  - [6.2 - 后向传播的线性激活](#6-2)
    - [练习 8 - 后向传播的线性激活](#ex-8)
  - [6.3 - L层模型的后向传播](#6-3)
    - [练习 9 - L层模型的后向传播](#ex-9)
  - [6.4 - 更新参数](#6-4)
    - [练习 10 - 更新参数](#ex-10)

<a name='1'></a>
## 1 - 导包

首先你需要导入任务中要用到的相关包

- [numpy](https://numpy.org/doc/1.20/):Python中用于科学计算的基础包
- [matplotlib](http://matplotlib.org)：Python中著名画图库
- dnn_utils 在这次任务中提供各种能帮到你的函数
- testCases 提供一些测试案例来评估你函数的正确性
- np.random.seed(1) 设置随机数seed，固定结果(方便测试)

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases import *
from dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward
from public_tests import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # 设定绘图大小
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

<a name='2'></a>
## 2 - 概要

为了搭建神经网络，你需要先实现几个辅助函数，这些辅助函数将会在后期搭建两层神经网络和L层神经网络用到。每个辅助函数都有详细的说明，指导你完成必要的步骤

以下是作业步骤的概要：

- 初始化二层(或L层)神经网络要用到的参数
- 实现前向传播模块(下图中的紫色部分)
     - 完成层中前向传播的线性部分(算出resulting in $Z^{[l]}$)
     - 激活函数已经提供(relu/sigmoid)
     - 将前两步结合成一个新的[LINEAR->ACTIVATION]前向函数
     - 把[LINEAR->RELU]前向叠加(从第1层到第L-1层)，然后在最后添加[LINEAR->SIGMOID]前向函数(第L层)，这时你会得到一个新的L层模型前向函数.
- 计算loss
- 实现后向传播模块(下图中的红色部分)
    - 完成层中后向传播的线性部分
    - 激活函数的梯度已经提供(relu_backward/sigmoid_backward) 
    - 将前两步结合成一个新的[LINEAR->ACTIVATION]后向函数
    - 把[LINEAR->RELU]后向叠加L-1次，最后后向叠加一次[LINEAR->SIGMOID]函数，这时你会得到一个新的L层模型后向函数
- 最终更新参数

<img src="images/final outline.png" style="width:800px;height:500px;">
<caption><center><b>图 1</b></center></caption><br>


**笔记**:

对于每一个前向传播函数，都会有与之匹配的后向函数。这就是为什么你需要在前向模块中的每一步都要存储缓存cache，缓存值在后期计算梯度中很有用

在后向传播模块中，你可以用刚才得到的cache计算梯度值。相关步骤本次作业都会带你一步一步地完成，不用担心

<a name='3'></a>
## 3 - 初始化

你将会编写两个辅助函数，这两个函数用来初始化模型中需要的参数
- 第一个函数用来初始化二层模型的参数
- 第二个函数用来归纳初始化过程(将参数整合起来)

<a name='3-1'></a>
### 3.1 - 2层神经网络

<a name='ex-1'></a>
### 练习 1 - 初始化参数

创建和初始化2层神经网络参数

**说明**:

- 模型的整体结构是: *LINEAR -> RELU -> LINEAR -> SIGMOID*. 
- 对权重矩阵进行随机初始化: `np.random.randn(shape)*0.01`，注意shape不要出错
- 对偏移向量进行0初始化: `np.zeros(shape)`

In [2]:
# 分级函数: 初始化参数

def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- 输入层大小
    n_h -- 隐藏层大小
    n_y -- 输出层大小
    
    Returns:
    parameters -- python字典，包含以下参数:
                    W1 -- weight，权重矩阵，shape为(n_h, n_x)
                    b1 -- bias，偏移向量，shape为(n_h, 1)
                    W2 -- weight，权重矩阵，shape为(n_y, n_h)
                    b2 -- bias，偏移向量，shape为(n_y, 1)
    """
    
    np.random.seed(1)
    
    #(预计 4 行代码完成)
    # W1 = ...
    # b1 = ...
    # W2 = ...
    # b2 = ...
    # 代码练习区-起始部分
    

    
    # 代码练习区-结束部分
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters    

In [3]:
parameters = initialize_parameters(3,2,1)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

initialize_parameters_test(initialize_parameters)

W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[0.]]
 测试全部通过


***期望输出***
```
W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[0.]]
```

<a name='3-2'></a>
### 3.2 - L层神经网络

L层神经网络的初始化更复杂，因为里面会有更多的权重矩阵和偏移向量

当实现`initialize_parameters_deep()`函数时，你应该确信你的维度要和每一层相匹配

回忆：$n^{[l]}$是第$l$层的单元数. 例如， 如果你的输入数据$X$维度是$(12288, 209)$ (with $m=209$) ，那么：

<table style="width:100%">
    <tr>
        <td>  </td> 
        <td> <b>W的shape</b> </td> 
        <td> <b>b的shape</b>  </td> 
        <td> <b>激活函数</b> </td>
        <td> <b>激活值的shape</b> </td> 
    <tr>
    <tr>
        <td> <b>第1层</b> </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    <tr>
        <td> <b>第2层</b> </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>  
   <tr>
       <td> <b>第L-1层</b> </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
   <tr>
   <tr>
       <td> <b>第L层</b> </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>
</table>

请记住，当你在python计算$W X + b$时会触发广播机制(boradcasting)

例：如果: 

$$ W = \begin{bmatrix}
    w_{00}  & w_{01} & w_{02} \\
    w_{10}  & w_{11} & w_{12} \\
    w_{20}  & w_{21} & w_{22} 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    x_{00}  & x_{01} & x_{02} \\
    x_{10}  & x_{11} & x_{12} \\
    x_{20}  & x_{21} & x_{22} 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    b_0  \\
    b_1  \\
    b_2
\end{bmatrix}\tag{2}$$

那么$WX + b$会变成:

$$ WX + b = \begin{bmatrix}
    (w_{00}x_{00} + w_{01}x_{10} + w_{02}x_{20}) + b_0 & (w_{00}x_{01} + w_{01}x_{11} + w_{02}x_{21}) + b_0 & \cdots \\
    (w_{10}x_{00} + w_{11}x_{10} + w_{12}x_{20}) + b_1 & (w_{10}x_{01} + w_{11}x_{11} + w_{12}x_{21}) + b_1 & \cdots \\
    (w_{20}x_{00} + w_{21}x_{10} + w_{22}x_{20}) + b_2 &  (w_{20}x_{01} + w_{21}x_{11} + w_{22}x_{21}) + b_2 & \cdots
\end{bmatrix}\tag{3}  $$


<a name='ex-2'></a>
### 练习 2 -  初始化参数深度

实现一个L层神经网络的初始化

**说明**:
- 模型的结构是 *[LINEAR -> RELU] $ \times$ (L-1) -> LINEAR -> SIGMOID*. I.e., 其中$L-1$层使用ReLU激活函数，最后的输出层使用sigmoid激活函数
- 对权重矩阵使用随机初始化(`np.random.randn(shape) * 0.01`)
- 对偏移向量使用0初始化(`np.zeros(shape)`)
- 你将会用变量`layer_dims`来存储不同层对应的单元数$n^{[l]}$，例如上周的二维数据分类模型对应的是 [2,4,1] (输入层有2个单元，有1个隐含层且隐含层有4个隐含单元，输出层有1个单元)，这也意味着`W1`的shape是(4,2)，`b1`的shape是(4,1)，`W2`的shape是(1,4)，`b2`的shape是(1,1)。现在你将会把这个概念推广到L层
- 这里是L=1，也就是1层神经网络的实现，应该会对你对通用情况(L层神经网络)下的实现有启发
```python
    if L == 1:
        parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
        parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```

In [4]:
# 分级函数: 初始化参数深度

def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python的array (list)它包含我们神经网络中每一层的维度
    
    Returns:
    parameters -- python的dictionary，它包含参数"W1", "b1", ..., "WL", "bL":
                    Wl -- 权重矩阵，shape为(layer_dims[l], layer_dims[l-1])
                    bl -- 偏移向量，shape为 (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # 网络层数

    for l in range(1, L):
        #(预计 2 行代码完成)
        # parameters['W' + str(l)] = ...
        # parameters['b' + str(l)] = ...
        # 代码练习区-起始部分
        

        
        # 代码练习区-结束部分
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [5]:
parameters = initialize_parameters_deep([5,4,3])

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

initialize_parameters_deep_test(initialize_parameters_deep)

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]
 测试全部通过


***期望输出***
```
W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]
```

<a name='4'></a>
## 4 - 前向传播模块

<a name='4-1'></a>
### 4.1 - 前向传播-线性部分

完成了参数初始化后你就可以继续完成前向传播模块了

首先我们先实现一些必要的辅助函数，用于后期模型的实现，现在将要按次序完成以下函数：

- LINEAR
- LINEAR -> ACTIVATION (ACTIVATION是ReLU或Sigmoid). 
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID (整个模型)

线性前向传播模块(所有样本向量化)计算公式如下：

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

其中 $A^{[0]} = X$. 

<a name='ex-3'></a>
### 练习 3 - 前向传播-线性部分

建立前向传播的线性部分


**提醒**:
数学公式$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$的实现可以借助 `np.dot()`函数.，记住用`W.shape`来检查维度是否正确

In [6]:
# 分级函数: 前向传播-线性部分

def linear_forward(A, W, b):
    """
    实现前向传播的线性部分

    Arguments:
    A -- 前一层(或输出层)的激活值: (上一层单元数, 样本数)
    W -- 权重矩阵: numpy array，shape为(当前层单元数, 上一层单元数)
    b -- 偏移向量, numpy array，shape为(当前层单元数, 1)

    Returns:
    Z -- 激活函数的输入值，也叫做预激活参数 
    cache -- python tuple，包含"A", "W" and "b" ; 存储后向传播要用到的参数
    """
    
    #(预计 1 行代码完成)
    # Z = ...
    # 代码练习区-起始部分
    

    
    # 代码练习区-结束部分
    cache = (A, W, b)
    
    return Z, cache

In [7]:
t_A, t_W, t_b = linear_forward_test_case()
t_Z, t_linear_cache = linear_forward(t_A, t_W, t_b)
print("Z = " + str(t_Z))

linear_forward_test(linear_forward)

Z = [[ 3.26295337 -1.23429987]]
 测试全部通过


***期望输出***
```
Z = [[ 3.26295337 -1.23429987]]
```

<a name='4-2'></a>
### 4.2 - 前向传播-激活部分

在本次练习中，你将会用到两个激活函数：

- **Sigmoid**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}} \ \$ 现在`sigmoid`函数已经给出，该函数有两个返回值：激活值`a`和`cache`(包含后向传播会用到的`z`)，以下是它的调用方法：
``` python
A, activation_cache = sigmoid(Z)
```

- **ReLU**: 数学定义为$A = RELU(Z) = max(0, Z)$. 现在`relu`函数已经给出，该函数有两个返回值：激活值`a`和`cache`(包含后向传播会用到的`z`)，以下是它的调用方法：
``` python
A, activation_cache = relu(Z)
```

为了方便，你可以把两个函数(Linear and Activation)合并成一个函数(LINEAR->ACTIVATION)

<a name='ex-4'></a>
### 练习 4 - 前向传播-线性+激活部分

该部分主要实现前向传播的 *LINEAR->ACTIVATION* 层，数学关联为$A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$

其中激活部分既可以是`sigmoid()`函数也可以是`relu()`函数. 

该函数需要借助`linear_forward()`函数和正确的激活函数

In [8]:
# 分级函数: 前向传播-线性+激活部分

def linear_activation_forward(A_prev, W, b, activation):
    """
    实现前向传播的LINEAR->ACTIVATION层

    Arguments:
    A_prev -- 上一层的激活值(或者是输入函数)(上一层单元数, 样本数)
    W -- 权重矩阵: numpy array，shape为(当前层单元数, 上一层单元数)
    b -- 偏移向量, numpy array，shape为(当前层单元数, 1)
    activation -- 这层要用到的激活函数 有两种string模式: "sigmoid" or "relu"

    Returns:
    A -- 激活函数输出值，也被叫做激活后的值
    cache -- 一个python的tuple, 包含"linear_cache" and "activation_cache"，
             后向传播计算时会用到
    """
    
    if activation == "sigmoid":
        #(预计 2 行代码完成)
        # Z, linear_cache = ...
        # A, activation_cache = ...
        # 代码练习区-起始部分
        

        
        # 代码练习区-结束部分
    
    elif activation == "relu":
        #(预计 2 行代码完成)
        # Z, linear_cache = ...
        # A, activation_cache = ...
        # 代码练习区-起始部分
        
        
        # 代码练习区-结束部分
    cache = (linear_cache, activation_cache)

    return A, cache

In [9]:
t_A_prev, t_W, t_b = linear_activation_forward_test_case()

t_A, t_linear_activation_cache = linear_activation_forward(t_A_prev, t_W, t_b, activation = "sigmoid")
print("With sigmoid: A = " + str(t_A))

t_A, t_linear_activation_cache = linear_activation_forward(t_A_prev, t_W, t_b, activation = "relu")
print("With ReLU: A = " + str(t_A))

linear_activation_forward_test(linear_activation_forward)

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]
 测试全部通过


***期望输出***
```
With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]
```

**注意**: 在深度学习中，"[LINEAR->ACTIVATION]"在神经网络中算一层，不是两层

<a name='4-3'></a>
### 4.3 - L层模型

为了在实现L层神经网络时更加方便，你将需要一个函数，这个函数需要复制RELU类型的`linear_activation_forward()`函数$L-1$次，然后使用一个SIGMOID类型的`linear_activation_forward()`函数

<img src="images/model_architecture_kiank.png" style="width:600px;height:300px;">
<caption><center> <b>Figure 2</b> : *[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* 模型</center></caption><br>

<a name='ex-5'></a>
### 练习 5 -  L层模型的前向传播

实现上图模型的前向传播

**说明**: 在下行代码中，变量`AL`代表$A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$ （有时候也会被叫做`Y`帽$\hat{Y}$）

**提示**:
- 使用你刚才编写的函数
- 使用for循环[LINEAR->RELU]复制，(L-1)次
- 不要忘记追踪`cache`列表中的缓存，你可以使用`list.append(c)`把 `c`尾插到`list`里

In [10]:
# 分级函数: L层模型的前向传播

def L_model_forward(X, parameters):
    """
    实现前向传播计算：[LINEAR->RELU]*(L-1)->LINEAR->SIGMOID
    
    Arguments:
    X -- 输入数据，numpy数组，shape为(输入层单元数, 样本数)
    parameters -- initialize_parameters_deep()函数的输出
    
    Returns:
    AL -- 输出层(也就是最后一层)的激活值
    caches -- 缓存列表，包含linear_activation_forward()函数中的缓存(用分L层，索引值从0到L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  #神经网络层数 3层
#     print(parameters['W1'].shape,parameters['W2'].shape,parameters['W3'].shape,parameters['b1'].shape,parameters['b2'].shape,parameters['b3'].shape)
#     测试案例对应的：(4, 5) (3, 4) (1, 3) (4, 1) (3, 1) (1, 1)
    # 实现[LINEAR -> RELU]*(L-1). 把"cache"添加到"caches"列表
    # 从层1开始循环，因为层0是输入层
    for l in range(1, L):
        A_prev = A 
        #(预计 2 行代码完成)
        # A, cache = ...
        # caches ...
        # 代码练习区-起始部分

        
        # 代码练习区-结束部分
    
    # 实现LINEAR -> SIGMOID. 把"cache"添加到"caches"列表
    #(预计 2 行代码完成)
    # AL, cache = ...
    # caches ...
    # 代码练习区-起始部分
    

    
    # 代码练习区-结束部分
          
    return AL, caches

In [11]:
t_X, t_parameters = L_model_forward_test_case_2hidden()
t_AL, t_caches = L_model_forward(t_X, t_parameters)

print("AL = " + str(t_AL))

L_model_forward_test(L_model_forward)

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
 测试全部通过


***期望输出***
```
AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
```

**真棒!** 你已经实现了一个完整的前向传播，接受输入X输出包含预测的$A^{[L]}$行向量。它将所有的中间值记录到`caches`，你可以用Using $A^{[L]}$计算你预测的cost

<a name='5'></a>
## 5 - 代价函数

现在你可以实现前向和后向传播了！为了检查你的模型是否真正在学习，你需要计算cost

<a name='ex-6'></a>
### 练习 6 - 计算cost

使用公式$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$ 计算交叉熵cost$J$

In [12]:
# GRADED FUNCTION: compute_cost

def compute_cost(AL, Y):
    """
    根据(7)式实现代价函数

    Arguments:
    AL -- 预测结果向量(原：probability vector corresponding to your label predictions), shape为(1, 样本数)
    Y -- 实际结果向量 (猫1非猫0), shape为(1, 样本数)

    Returns:
    cost -- 交叉熵cost
    """
    
    m = Y.shape[1]

    # 根据aL和y计算cost
    # (预计 1 行代码完成)
    # cost = ...
    # 代码练习区-起始部分
    

    
    # 代码练习区-结束部分
    
    cost = np.squeeze(cost)      # 注意shape要正确 (e.g. this turns [[17]] into 17).

    
    return cost

In [13]:
t_Y, t_AL = compute_cost_test_case()
t_cost = compute_cost(t_AL, t_Y)

print("Cost: " + str(t_cost))

compute_cost_test(compute_cost)

Cost: 0.2797765635793422
 测试全部通过


**期望输出**:

<table>
    <tr>
        <td><b>cost</b> </td>
    <td> 0.2797765635793422</td> 
    </tr>
</table>

<a name='6'></a>
## 6 - 后向传播模块

像前向传播一样，在实现后向传播前你需要实现集合辅助函数，请记住后向传播是用来计算损失函数相对于参数的梯度

**暗示**: 
<img src="images/backprop_kiank.png" style="width:650px;height:250px;">
<caption><center><font color='purple'><b>图 3</b>: LINEAR->RELU->LINEAR->SIGMOID 的前向传播(紫色)与后向传播(红色) </font></center></caption>


**解释(MD注释部分)**

对于你们这样的计算专家(你不需要完成这项任务)，链式求导法则可以被用来计算$\mathcal{L}$对$z^{[1]}$的导数，比如2层网络的计算就是 the chain rule of calculus can be used to derive:

$$\frac{d \mathcal{L}(a^{[2]},y)}{{dz^{[1]}}} = \frac{d\mathcal{L}(a^{[2]},y)}{{da^{[2]}}}\frac{{da^{[2]}}}{{dz^{[2]}}}\frac{{dz^{[2]}}}{{da^{[1]}}}\frac{{da^{[1]}}}{{dz^{[1]}}} \tag{8} $$

为了计算$dW^{[1]} = \frac{\partial L}{\partial W^{[1]}}$,你可以根据链式求导法则得到$dW^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial W^{[1]}}$. 
在后向传播时， 在每一步，你用你当前的梯度乘以特定层所对应的梯度，得到你想要的梯度。(原文：During backpropagation, at each step you multiply your current gradient by the gradient corresponding to the specific layer to get the gradient you wanted.)

为了计算梯度$db^{[1]} = \frac{\partial L}{\partial b^{[1]}}$, 你可以根据链式求导法则得到$db^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial b^{[1]}}$.

这就是为什么我们要讨论 **反向传播**.

--

现在和前向传播相似，你要通过以下三步实现反向传播：

1. LINEAR backward
2. LINEAR -> ACTIVATION backward (其中ACTIVATION是ReLU或sigmoid激活函数)
3. [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID backward (整个模型)

为了下个练习，你需要记住:

- `b`是一个n行1列的矩阵(np.ndarray), i.e: b = [[1.0], [2.0]] (请记住`b`是一个常量)
- np.sum对数组的元素进行求和
- axis=1表示按行求和(跨列)，axis=0表示按列求和(跨行)
- keepdims指定是否必须保留矩阵的原始尺寸
- 下例可以帮助你理解内容

In [14]:
A = np.array([[1, 2],
              [3, 4]])

print('axis=1 and keepdims=True')
print(np.sum(A, axis=1, keepdims=True))
print('axis=1 and keepdims=False')
print(np.sum(A, axis=1, keepdims=False))
print('axis=0 and keepdims=True')
print(np.sum(A, axis=0, keepdims=True))
print('axis=0 and keepdims=False')
print(np.sum(A, axis=0, keepdims=False))

axis=1 and keepdims=True
[[3]
 [7]]
axis=1 and keepdims=False
[3 7]
axis=0 and keepdims=True
[[4 6]]
axis=0 and keepdims=False
[4 6]


<a name='6-1'></a>
### 6.1 - 线性后向传播

$l$层的线性部分为: $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (然后在激活).

假定你已经计算了导数 $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$. 你需要得到 $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$.

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption><center><font color='purple'><b>图 4</b></font></center></caption>

用$dZ^{[l]}$计算输出$(dW^{[l]}, db^{[l]}, dA^{[l-1]})$三个变量


附上相关公式:
$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


$A^{[l-1] T}$是$A^{[l-1]}$的转置. 

<a name='ex-7'></a>
### 练习 7 - 线性后向传播

用上面三个公式是实现`linear_backward()`函数

**提示**:

- 你可以使用`A.T`或`A.transpose()`实现`A`的转置

In [15]:
# 分级函数: 线性后向传播

def linear_backward(dZ, cache):
    """
    实现第l层后向传播的线性部分

    Arguments:
    dZ -- l层成本相对于线性输出的梯度
    cache -- python的tuple(A_prev, W, b)在当前层的后向传播

    Returns:
    dA_prev -- 相对于激活值的成本梯度 (l-1层), 和A_prev的shape相同
    dW -- 相对于W的成本梯度 (l层), 和W的shape相同
    db -- 相对于b的成本梯度 (l层), 和b的shape相同
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### 代码区 ### (预计 3 行代码完成)
    # dW = ...
    # db = ... 按dz行求和,keepdims=True
    # dA_prev = ...
    # 代码练习区-起始部分
    

    
    # 代码练习区-结束部分
    
    return dA_prev, dW, db

In [16]:
t_dZ, t_linear_cache = linear_backward_test_case()
t_dA_prev, t_dW, t_db = linear_backward(t_dZ, t_linear_cache)

print("dA_prev: " + str(t_dA_prev))
print("dW: " + str(t_dW))
print("db: " + str(t_db))

linear_backward_test(linear_backward)

dA_prev: [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW: [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db: [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]
 测试全部通过


**期望输出**:
```
dA_prev: [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW: [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db: [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]
 ```

<a name='6-2'></a>
### 6.2 - 后向传播的线性激活

之后你将会创建一个函数，将两个辅助函数`linear_backward()`函数和`linear_activation_backward()`函数

为了帮助您实现`linear_activation_backward()`函数，现在已经提供了两个函数

- **`sigmoid_backward`**: 实现SIGMOID单元的后向传播过程，调用方式如下：

```python
dZ = sigmoid_backward(dA, activation_cache)
```

- **`relu_backward`**: 实现RELU单元的后向传播过程，调用方式如下：

```python
dZ = relu_backward(dA, activation_cache)
```

如果$g(.)$是激活函数，使用`sigmoid_backward`和`relu_backward`计算的方法如下：$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}). \tag{11}$$  

<a name='ex-8'></a>
### 练习 8 -  后向传播的线性激活

实现*LINEAR->ACTIVATION*层的后向传播

In [17]:
# 分级函数: 后向传播的线性激活

def linear_activation_backward(dA, cache, activation):
    """
    实现LINEAR->ACTIVATION层的后向传播
    
    Arguments:
    dA -- 当前l层的激活梯度
    cache -- tuple，值为(linear_cache, activation_cache)，用于后向传播
    activation -- 该层的激活函数："sigmoid" or "relu"
    
    Returns:
    dA_prev -- 相对于激活值的成本梯度 (l-1层), 和A_prev的shape相同
    dW -- 相对于W的成本梯度 (l层), 和W的shape相同
    db -- 相对于b的成本梯度 (l层), 和b的shape相同
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        #(预计 2 行代码完成)
        # dZ =  ...
        # dA_prev, dW, db =  ...
        # 代码练习区-起始部分
        

        
        # 代码练习区-结束部分
        
    elif activation == "sigmoid":
        #(预计 2 行代码完成)
        # dZ =  ...
        # dA_prev, dW, db =  ...
        # 代码练习区-起始部分
        

        
        # 代码练习区-结束部分
    
    return dA_prev, dW, db

In [18]:
t_dAL, t_linear_activation_cache = linear_activation_backward_test_case()

t_dA_prev, t_dW, t_db = linear_activation_backward(t_dAL, t_linear_activation_cache, activation = "sigmoid")
print("With sigmoid: dA_prev = " + str(t_dA_prev))
print("With sigmoid: dW = " + str(t_dW))
print("With sigmoid: db = " + str(t_db))

t_dA_prev, t_dW, t_db = linear_activation_backward(t_dAL, t_linear_activation_cache, activation = "relu")
print("With relu: dA_prev = " + str(t_dA_prev))
print("With relu: dW = " + str(t_dW))
print("With relu: db = " + str(t_db))

linear_activation_backward_test(linear_activation_backward)

With sigmoid: dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
With sigmoid: dW = [[ 0.10266786  0.09778551 -0.01968084]]
With sigmoid: db = [[-0.05729622]]
With relu: dA_prev = [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]]
With relu: dW = [[ 0.44513824  0.37371418 -0.10478989]]
With relu: db = [[-0.20837892]]
 测试全部通过


<a name='6-3'></a>
### 6.3 - L层模型的后向传播

现在你将要实现整个网络的后向传播函数

回忆以下你实现`L_model_forward()`函数的适合，每一次迭代你都会有一个缓存(X,W,b,和 z)

在后向传播模块，你将要使用这些变量去计算梯度，因此在`L_model_backward()`函数，你将会从$L$层向后遍历所有的隐藏层，每一步你都会使用$l$层的缓存值进行$l$层后向传播


<img src="images/mn_backward.png" style="width:450px;height:300px;">
<caption><center><font color='purple'><b>图 5</b>: 后向传播的过程</font></center></caption>

**后向传播的初始化**:

为了让整个网络进行后向传播，你知道输出值$A^{[L]} = \sigma(Z^{[L]})$，所以你需要去计算 `dAL` $= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$，代码实现如下：

```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # cost对于AL的导数
```

然后你就可以使用激活后梯度`dAL`来进行后向传播，在图五，你现在可以将`dAL`用到你要实现的LINEAR->SIGMOID后向传播部分(该过程将使用`L_model_forward()`函数进行缓存存储)

在那之后，你将会用到for循环对其他的层进行LINEAR->RELU后向传播操作，你应该在梯度字典里存储每层的dA, dW和db。以下是可能会用到的公式

$$grads["dW" + str(l)] = dW^{[l]}\tag{15} $$

例如，当$l=3$时，应该将$dW^{[l]}$存储至`grads["dW3"]`.

<a name='ex-9'></a>
### 练习 9 -  L层模型的后向传播

实现*[LINEAR->RELU] $\times$ (L-1) -> LINEAR -> SIGMOID*模型的后向传播

In [19]:
# 分级函数: L层模型的后向传播

def L_model_backward(AL, Y, caches):
    """
    实现*[LINEAR->RELU]  ×  (L-1) -> LINEAR -> SIGMOID*模型的后向传播
    
    Arguments:
    AL -- 可能性向量, 前向传播的输出 (L_model_forward())
    Y -- 实际值向量 (猫1非猫0)
    caches -- caches列表，包含:
                linear_activation_forward()每一个cache
                其中：relu：it's caches[l], for l in range(L-1) i.e l = 0...L-2
                      sigmoid：it's caches[L-1]
                      
    Returns:
    grads -- python字典，包含梯度
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # 神经网络的层数
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # 将Y的shape和AL的shape一致
    
    # 初始化后向传播
    #(预计 1 行代码完成)
    # dAL = ...
    # 代码练习区-起始部分
    

    
    # 代码练习区-结束部分
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    #(预计 5 行代码完成)
    # current_cache = ...
    # dA_prev_temp, dW_temp, db_temp = ...
    # grads["dA" + str(L-1)] = ...
    # grads["dW" + str(L)] = ...
    # grads["db" + str(L)] = ...
    # 代码练习区-起始部分
    

    
    # 代码练习区-结束部分
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        #(预计 5 行代码完成)
        # current_cache = ...
        # dA_prev_temp, dW_temp, db_temp = ...
        # grads["dA" + str(l)] = ...
        # grads["dW" + str(l + 1)] = ...
        # grads["db" + str(l + 1)] = ...
        # 代码练习区-起始部分
        
        
        # 代码练习区-结束部分

    return grads

In [20]:
t_AL, t_Y_assess, t_caches = L_model_backward_test_case()
grads = L_model_backward(t_AL, t_Y_assess, t_caches)

print("dA0 = " + str(grads['dA0']))
print("dA1 = " + str(grads['dA1']))
print("dW1 = " + str(grads['dW1']))
print("dW2 = " + str(grads['dW2']))
print("db1 = " + str(grads['db1']))
print("db2 = " + str(grads['db2']))

L_model_backward_test(L_model_backward)

dA0 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]
dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
dW2 = [[-0.39202432 -0.13325855 -0.04601089]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
db2 = [[0.15187861]]
 测试全部通过


**期望输出：**

```
dA0 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]
dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
dW2 = [[-0.39202432 -0.13325855 -0.04601089]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
db2 = [[0.15187861]]
```

<a name='6-4'></a>
### 6.4 - 更新参数

在这节，你将会用梯度下降更新模型参数：

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

其中 $\alpha$是学习率 

更新参数后记得将他们存储到`parameters()`字典里

<a name='ex-10'></a>
### 练习 10 - 更新参数

完成`update_parameters()`，通过梯度下降更新你的参数

**提示**:
要更新每一个$W^{[l]}$和$b^{[l]}$ ($l = 1, 2, ..., L$)的参数

In [21]:
# 分级函数: 更新参数

def update_parameters(params, grads, learning_rate):
    """
    通过梯度下降更新参数
    
    Arguments:
    params -- python字典，包含你的参数
    grads -- python字典，包含你的梯度，即L_model_backward的输出
    
    Returns:
    parameters -- python字典，包含你更新的参数
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    parameters = params.copy()
    L = len(parameters) // 2 # 神经网络的层数

    # 使用循环更新每个参数
    #(预计 2 行代码完成)
    for l in range(L):
        # parameters["W" + str(l+1)] = ...
        # parameters["b" + str(l+1)] = ...
        # 代码练习区-起始部分
        

        
        # 代码练习区-结束部分
    return parameters

In [22]:
t_parameters, grads = update_parameters_test_case()
t_parameters = update_parameters(t_parameters, grads, 0.1)

print ("W1 = "+ str(t_parameters["W1"]))
print ("b1 = "+ str(t_parameters["b1"]))
print ("W2 = "+ str(t_parameters["W2"]))
print ("b2 = "+ str(t_parameters["b2"]))

update_parameters_test(update_parameters)

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
 测试全部通过


**期望输出：**

```
W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
```

### 恭喜你! 

你刚才完成了所有建立深度神经网络用到的函数，其中包括：

- 使用非线性单元改善模型
- 建立一个多层神经网络(隐含层数大于1)
- 实现一个易用的神经网络类

这次的任务确实有些困难，但下任务会简单很多;) 

在下个任务中，你将会把这些函数合并成两个模型

- 2层神经网络
- L层神经网络

事实上，你可以使用这些模型来识别图像是否为猫！做的不错，下次我们再见！